In [2]:
import pandas as pd
import re, os
import matplotlib.pyplot as plt
import numpy as np

In [1]:
# load original data from file obtained at https://www.icpsr.umich.edu/web/NACJD/studies/38492/datadocumentation#
df = pd.read_csv("/Users/paritashah/Desktop/College/Year3/Term2/BigData/finalproject/CorrectionalAnalysis/admissions.tsv", sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/paritashah/Desktop/College/Year3/Term2/BigData/finalproject/CorrectionalAnalysis/admissions.tsv'

In [2]:
# function to clean up the data, remove the rows with missing data
def remove_missing(df):
    # remove unknown education levels
    df = df[df.EDUCATION != 9]
    # remove unkown and other prison admission types
    df = df[df.ADMTYPE != 9]
    df = df[df.ADMTYPE != 3]
    # remove unkonwn general offense categories
    df = df[df.OFFGENERAL != 9]
    # drop columns with mostly missing information 
    df = df.drop(columns=['MAND_PRISREL_YEAR', 'PROJ_PRISREL_YEAR', 'PARELIG_YEAR'])
    # drop report year column (we are primarily interested in the admission year)
    df = df.drop(columns=['RPTYEAR'])
    # remove unknown admission year datapoints
    df = df[df.ADMITYR != 9999]
    # remove admission type column (may be of interest later, but not current focus)
    df = df.drop(columns=['ADMTYPE'])
    # remove unknown maximum sentence length datapoints
    df = df[df.SENTLGTH != 9]
    # remove unknown offense detail datapoints
    df = df[df.OFFDETAIL != 99]
    # remove unknown race datapoints
    df = df[df.RACE != 9]
    # remove unknown age at admission datapoints 
    df = df[df.AGEADMIT != 9]

    return df

In [3]:
# save dataframe as csv
df = remove_missing(df)
df.to_csv('PrisonAdmissionsCleaned.csv', index=False)

In [5]:
fips_codes = [1, 2, 4, 5, 6, 8, 9, 11, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40, 42, 44, 45, 46, 47, 48, 49, 51, 53, 54, 55, 56]
state_codes = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "FL", "GA", "HI", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NY", "NC", "ND", "OH", "OK", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "WA", "WV", "WI", "WY"]
detailed_offenses = ["murder", "negligent manslaugher", "rape/SA", "robbery", "assault", "other violent", "burglary", "larceny", "motor vehicle theft", "fraud", "other property", "drugs", "public order", "unspecified"]
races = ["white", "black", "hispanic", "other"]

In [4]:
# convert the dataframe to a one hot encoded dataframe
df = pd.read_csv('PrisonAdmissionsCleaned.csv')

# create new columns for one-hot encoding
states_col = pd.get_dummies(state_codes, dtype=int)
races_col = pd.get_dummies(races, dtype=int)
offense_col = pd.get_dummies(detailed_offenses, dtype=int)

df = pd.concat([df, states_col, races_col, offense_col])

# for each state code in state_codes, add a column to the dataframe with the state code
for index, state in enumerate(state_codes):
    code = fips_codes[index]
    # if the STATE column contains an FIPS code corresponding to the name of a state, add a 1 in that state's column
    df[state] = df.apply(lambda row: 1 if row['STATE'] == code else 0, axis=1)

# for each offense in detailed_offenses, add a column to the dataframe
for index, race in enumerate(races):
    code = index + 1
    df[race] = df.apply(lambda row: 1 if row['RACE'] == code else 0, axis=1)

# for each race, add a column to the dataframe
for index, offense in enumerate(detailed_offenses):
    code = index + 1
    df[offense] = df.apply(lambda row: 1 if row['OFFDETAIL'] == code else 0, axis=1)

# df.drop(columns=['RACE', 'STATE', 'OFFDETAIL'], axis=1)

# save the dataframe to a csv file as oneHotDfAllCols.csv
df.to_csv('PrisonAdmissionsOneHotEncoding.csv')


In [5]:
df.to_csv('PrisonAdmissionsCleanOneHotEncoding.csv', index=False)

In [3]:
# read csv, drop encoded columns
one_hot_df = pd.read_csv("Data/allData/PrisonAdmissionsCleanOneHotEncoding.csv") 
one_hot_df.drop(columns=['STATE', 'RACE', 'OFFDETAIL'], inplace=True)
one_hot_df.head()

,SEX,EDUCATION,OFFGENERAL,ADMITYR,SENTLGTH,AGEADMIT,AK,AL,AR,AZ,...,larceny,motor vehicle theft,murder,negligent manslaugher,other property,other violent,public order,rape/SA,robbery,unspecified
0,1.0,2.0,2.0,1991.0,2.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1.0,2.0,1991.0,0.0,1.0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.0,2.0,4.0,1991.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1.0,1.0,2.0,1991.0,3.0,1.0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1.0,2.0,2.0,1991.0,2.0,2.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# create a new csv file from oneHotDfAllCols.csv only inlcuding data from states we are interested in 
one_hot_df_texas = one_hot_df.copy()
one_hot_df_texas = one_hot_df_texas[one_hot_df_texas.TX == 1]
one_hot_df_texas.drop(columns=state_codes, inplace=True)
one_hot_df_texas.to_csv('Data/stateData/oneHotDfTexas.csv', index=False)

one_hot_df_illinois = one_hot_df.copy()
one_hot_df_illinois = one_hot_df_illinois[one_hot_df_illinois.IL == 1]
one_hot_df_illinois.drop(columns=state_codes, inplace=True)
one_hot_df_illinois.to_csv('Data/stateData/oneHotDfIllinois.csv', index=False)

one_hot_df_florida = one_hot_df.copy()
one_hot_df_florida = one_hot_df_florida[one_hot_df_florida.FL == 1]
one_hot_df_florida.drop(columns=state_codes, inplace=True)
one_hot_df_florida.to_csv('Data/stateData/oneHotDfFlorida.csv', index=False)

one_hot_df_newyork = one_hot_df.copy()
one_hot_df_newyork = one_hot_df_newyork[one_hot_df_newyork.NY == 1]
one_hot_df_newyork.drop(columns=state_codes, inplace=True)
one_hot_df_newyork.to_csv('Data/stateData/oneHotDfNewYork.csv', index=False)

one_hot_df_nc = one_hot_df.copy()
one_hot_df_nc = one_hot_df_nc[one_hot_df_nc.NC == 1]
one_hot_df_nc.drop(columns=state_codes, inplace=True)
one_hot_df_nc.to_csv('Data/stateData/oneHotDfNC.csv', index=False)

one_hot_df_ca = one_hot_df.copy()
one_hot_df_ca = one_hot_df_ca[one_hot_df_ca.CA == 1]
one_hot_df_ca.drop(columns=state_codes, inplace=True)
one_hot_df_ca.to_csv('Data/stateData/oneHotDfCA.csv', index=False)

one_hot_df_az = one_hot_df.copy()
one_hot_df_az = one_hot_df_az[one_hot_df_az.AZ == 1]
one_hot_df_az.drop(columns=state_codes, inplace=True)
one_hot_df_az.to_csv('Data/stateData/oneHotDfAZ.csv', index=False)

one_hot_df_sc = one_hot_df.copy()
one_hot_df_sc = one_hot_df_sc[one_hot_df_sc.SC == 1]
one_hot_df_sc.drop(columns=state_codes, inplace=True)
one_hot_df_sc.to_csv('Data/stateData/oneHotDfSC.csv', index=False)

one_hot_df_mo = one_hot_df.copy()
one_hot_df_mo = one_hot_df_mo[one_hot_df_mo.MO == 1]
one_hot_df_mo.drop(columns=state_codes, inplace=True)
one_hot_df_mo.to_csv('Data/stateData/oneHotDfMO.csv', index=False)

one_hot_df_pa = one_hot_df.copy()
one_hot_df_pa = one_hot_df_pa[one_hot_df_pa.PA == 1]
one_hot_df_pa.drop(columns=state_codes, inplace=True)
one_hot_df_pa.to_csv('Data/stateData/oneHotDfPA.csv', index=False)

one_hot_df_in = one_hot_df.copy()
one_hot_df_in = one_hot_df_in[one_hot_df_in.IN == 1]
one_hot_df_in.drop(columns=state_codes, inplace=True)
one_hot_df_in.to_csv('Data/stateData/oneHotDfIN.csv', index=False)